In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import sys
sys.path.append(f'/media/beans/ssd/bespoke')

from models import EffNet
from constants import *
from imports import *
from rw_dataloader import RealWorldDataloader
from blender_dataloader import BlenderDataloader
from train_utils import *

torch.__version__, torch.cuda.device_count(), torch.cuda.get_device_name(torch.cuda.current_device())

/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:689: FutureWarning: This class has been deprecated. Please use CoarseDropout
  warnings.warn(
/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:965: FutureWarning: This class has been deprecated. Please use ImageCompression
  warnings.warn(


('1.10.0', 2, 'NVIDIA GeForce RTX 3090')

In [2]:
import wandb
run_id = wandb.util.generate_id(); run_id

'1eu39iy2'

In [3]:
wandb.init(id='1eu39iy2', project="carla", resume="allow")

wandb: Currently logged in as: rgilman33 (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.21 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [4]:
m = EffNet(model_arch="efficientnet_b3").to(device) # 13M params, 11.6M without RNN, 
sum([torch.numel(p) for p in m.parameters()]) / 1000

13502.597

In [7]:
sum([torch.numel(p) for p in m.backbone.parameters()]) / 1000

10696.232

wandb: Network error (ReadTimeout), entering retry loop.
wandb: Network error (ReadTimeout), entering retry loop.


In [5]:
#m.load_state_dict(torch.load(f"{BESPOKE_ROOT}/models/m.torch"), strict=False)

<All keys matched successfully>

In [6]:
m

EffNet(
  (backbone): EfficientNet(
    (conv_stem): Conv2d(3, 40, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act1): SiLU(inplace=True)
    (blocks): Sequential(
      (0): Sequential(
        (0): DepthwiseSeparableConv(
          (conv_dw): Conv2d(40, 40, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=40, bias=False)
          (bn1): BatchNorm2d(40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act1): SiLU(inplace=True)
          (se): SqueezeExcite(
            (conv_reduce): Conv2d(40, 10, kernel_size=(1, 1), stride=(1, 1))
            (act1): SiLU(inplace=True)
            (conv_expand): Conv2d(10, 40, kernel_size=(1, 1), stride=(1, 1))
            (gate): Sigmoid()
          )
          (conv_pw): Conv2d(40, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn2): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True

In [7]:
scaler = torch.cuda.amp.GradScaler() 
opt = torch.optim.Adam(m.parameters(), lr=5e-5)

In [8]:
BS = 32
dataloader_trn = BlenderDataloader(path_stem="trn", bs=BS)

In [9]:
%%time
#rw_dataloader = RealWorldDataloader()

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 6.44 µs


In [10]:
#wandb.watch(m, log="all")

In [11]:
#train_only_parts_of_model(m, part_to_train="fcs_2")

In [12]:
model_stem = "7.29"
START_E = 0

In [13]:
n_epochs = 1000
log_wandb = True

for e in range(START_E, n_epochs):
    
    opt.param_groups[0]['lr'] = 6e-4 if e < 50 else 3e-4 if e < 100 else 1e-4
    
    # Train
    print(f"EPOCH {e}\n")
    run_epoch(dataloader_trn, m, opt=opt, scaler=scaler, 
              train=True, backwards=True, log_wandb=log_wandb, wandb=wandb, updates_per_epoch=1280)
    
    print("\n\n Eval...")
    
    #eval_rw(rw_dataloader, m, wandb) # will log directly to wandb
        
    # Saving model checkpoint each epoch
    torch.save(m.state_dict(), f"{BESPOKE_ROOT}/models/m_{model_stem}_e{e}.torch")
    

EPOCH 0

{'trn_control_loss': 0.02232206, 'consistency losses/trn_steer_cost': 0.03980374, 'consistency losses/trn_te_loss': 20.07572608, 'consistency losses/trn_torque_loss': 556.50046006, 'consistency losses/trn_torque_delta_loss': 119.29560477, 'logistical/obs_consumed_per_second': 239.40944882, 'logistical/obs_generated_per_second': 111.18110236, 'logistical/data_consumption_ratio': 2.15627407, 'logistical/manual_train_pause': 0.0, 'logistical/max_param': 16.15359497, 'logistical/lr': 0.0003}
{'logistical/obs_consumed_per_second': 232.6796875, 'logistical/obs_generated_per_second': 113.59375, 'logistical/data_consumption_ratio': 2.05274351, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.02636462, 'consistency losses/trn_steer_cost': 0.03854609, 'consistency losses/trn_te_loss': 20.04642388, 'consistency losses/trn_torque_loss': 578.1319198, 'consistency losses/trn_torque_delta_loss': 120.04626116, 'logistical/max_param': 16.15159607, 'logistical/lr': 0.0003}
{'logistic

{'logistical/obs_consumed_per_second': 239.7109375, 'logistical/obs_generated_per_second': 111.7578125, 'logistical/data_consumption_ratio': 2.14633323, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.02324754, 'consistency losses/trn_steer_cost': 0.03911632, 'consistency losses/trn_te_loss': 19.69640729, 'consistency losses/trn_torque_loss': 588.15257862, 'consistency losses/trn_torque_delta_loss': 125.43564278, 'logistical/max_param': 16.15672302, 'logistical/lr': 0.0003}
{'logistical/obs_consumed_per_second': 240.765625, 'logistical/obs_generated_per_second': 109.015625, 'logistical/data_consumption_ratio': 2.21045837, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.02555752, 'consistency losses/trn_steer_cost': 0.04117882, 'consistency losses/trn_te_loss': 19.52620304, 'consistency losses/trn_torque_loss': 449.43241201, 'consistency losses/trn_torque_delta_loss': 119.46626318, 'logistical/max_param': 16.15864182, 'logistical/lr': 0.0003}
{'logistical/obs_con

{'logistical/obs_consumed_per_second': 236.765625, 'logistical/obs_generated_per_second': 113.625, 'logistical/data_consumption_ratio': 2.08782463, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.02610016, 'consistency losses/trn_steer_cost': 0.04037511, 'consistency losses/trn_te_loss': 20.26280616, 'consistency losses/trn_torque_loss': 537.66000797, 'consistency losses/trn_torque_delta_loss': 124.38013577, 'logistical/max_param': 16.1460495, 'logistical/lr': 0.0003}
{'logistical/obs_consumed_per_second': 236.7578125, 'logistical/obs_generated_per_second': 110.734375, 'logistical/data_consumption_ratio': 2.14161268, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.02258736, 'consistency losses/trn_steer_cost': 0.03798127, 'consistency losses/trn_te_loss': 20.72762114, 'consistency losses/trn_torque_loss': 552.26195008, 'consistency losses/trn_torque_delta_loss': 126.10414821, 'logistical/max_param': 16.1465435, 'logistical/lr': 0.0003}
{'logistical/obs_consumed_

{'logistical/obs_consumed_per_second': 162.0, 'logistical/obs_generated_per_second': 105.0, 'logistical/data_consumption_ratio': 1.55089206, 'logistical/manual_train_pause': 0.0}


 Eval...
EPOCH 5

{'trn_control_loss': 0.03339279, 'consistency losses/trn_steer_cost': 0.04048407, 'consistency losses/trn_te_loss': 19.5724972, 'consistency losses/trn_torque_loss': 615.52057622, 'consistency losses/trn_torque_delta_loss': 139.73344868, 'logistical/obs_consumed_per_second': 241.22834646, 'logistical/obs_generated_per_second': 103.91338583, 'logistical/data_consumption_ratio': 2.321277, 'logistical/manual_train_pause': 0.0, 'logistical/max_param': 16.1429882, 'logistical/lr': 0.0003}
{'logistical/obs_consumed_per_second': 233.8125, 'logistical/obs_generated_per_second': 106.921875, 'logistical/data_consumption_ratio': 2.19012857, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.03268623, 'consistency losses/trn_steer_cost': 0.04515505, 'consistency losses/trn_te_loss': 19.8194535

{'logistical/obs_consumed_per_second': 242.5546875, 'logistical/obs_generated_per_second': 105.8203125, 'logistical/data_consumption_ratio': 2.29243512, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.0404138, 'consistency losses/trn_steer_cost': 0.0459404, 'consistency losses/trn_te_loss': 19.93055169, 'consistency losses/trn_torque_loss': 595.89339662, 'consistency losses/trn_torque_delta_loss': 144.90256482, 'logistical/max_param': 16.14530754, 'logistical/lr': 0.0003}
{'logistical/obs_consumed_per_second': 231.4140625, 'logistical/obs_generated_per_second': 107.2578125, 'logistical/data_consumption_ratio': 2.16053923, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.02719492, 'consistency losses/trn_steer_cost': 0.03584427, 'consistency losses/trn_te_loss': 19.90809721, 'consistency losses/trn_torque_loss': 563.72139908, 'consistency losses/trn_torque_delta_loss': 146.71943456, 'logistical/max_param': 16.14574051, 'logistical/lr': 0.0003}
{'logistical/obs_con

...

{'logistical/obs_consumed_per_second': 232.2421875, 'logistical/obs_generated_per_second': 100.0859375, 'logistical/data_consumption_ratio': 2.32204323, 'logistical/manual_train_pause': 0.00070312, 'trn_control_loss': 0.01885849, 'consistency losses/trn_steer_cost': 0.04798192, 'consistency losses/trn_te_loss': 19.72004128, 'consistency losses/trn_torque_loss': 567.78500584, 'consistency losses/trn_torque_delta_loss': 123.55176932, 'logistical/max_param': 16.16024017, 'logistical/lr': 0.0001}
{'logistical/obs_consumed_per_second': 235.9609375, 'logistical/obs_generated_per_second': 100.28125, 'logistical/data_consumption_ratio': 2.35386014, 'logistical/manual_train_pause': 0.00015625, 'trn_control_loss': 0.01710382, 'consistency losses/trn_steer_cost': 0.04218316, 'consistency losses/trn_te_loss': 19.72577597, 'consistency losses/trn_torque_loss': 621.45843938, 'consistency losses/trn_torque_delta_loss': 125.33733761, 'logistical/max_param': 16.16005707, 'logistical/lr': 0.0001}
{'logi

{'trn_control_loss': 0.01770219, 'consistency losses/trn_steer_cost': 0.04921925, 'consistency losses/trn_te_loss': 20.15101524, 'consistency losses/trn_torque_loss': 474.1012144, 'consistency losses/trn_torque_delta_loss': 116.54914296, 'logistical/obs_consumed_per_second': 236.97637795, 'logistical/obs_generated_per_second': 105.43307087, 'logistical/data_consumption_ratio': 2.24686521, 'logistical/manual_train_pause': 0.0, 'logistical/max_param': 16.16109467, 'logistical/lr': 0.0001}
{'logistical/obs_consumed_per_second': 230.0546875, 'logistical/obs_generated_per_second': 102.5546875, 'logistical/data_consumption_ratio': 2.24470099, 'logistical/manual_train_pause': 0.00046875, 'trn_control_loss': 0.01538521, 'consistency losses/trn_steer_cost': 0.04011011, 'consistency losses/trn_te_loss': 19.86391775, 'consistency losses/trn_torque_loss': 556.61592808, 'consistency losses/trn_torque_delta_loss': 126.55310473, 'logistical/max_param': 16.1605835, 'logistical/lr': 0.0001}
{'logistica

{'logistical/obs_consumed_per_second': 234.5234375, 'logistical/obs_generated_per_second': 105.6875, 'logistical/data_consumption_ratio': 2.22367302, 'logistical/manual_train_pause': 7.812e-05, 'trn_control_loss': 0.01856655, 'consistency losses/trn_steer_cost': 0.04212511, 'consistency losses/trn_te_loss': 20.21077913, 'consistency losses/trn_torque_loss': 456.20880716, 'consistency losses/trn_torque_delta_loss': 116.20757198, 'logistical/max_param': 16.16044998, 'logistical/lr': 0.0001}
{'logistical/obs_consumed_per_second': 238.046875, 'logistical/obs_generated_per_second': 101.1953125, 'logistical/data_consumption_ratio': 2.3543284, 'logistical/manual_train_pause': 0.00054687, 'trn_control_loss': 0.01965851, 'consistency losses/trn_steer_cost': 0.04505622, 'consistency losses/trn_te_loss': 19.88498865, 'consistency losses/trn_torque_loss': 532.8315739, 'consistency losses/trn_torque_delta_loss': 119.4646126, 'logistical/max_param': 16.16095161, 'logistical/lr': 0.0001}
{'logistical

{'logistical/obs_consumed_per_second': 230.8203125, 'logistical/obs_generated_per_second': 99.890625, 'logistical/data_consumption_ratio': 2.31030993, 'logistical/manual_train_pause': 0.00054688, 'trn_control_loss': 0.01536727, 'consistency losses/trn_steer_cost': 0.03533161, 'consistency losses/trn_te_loss': 19.61468253, 'consistency losses/trn_torque_loss': 510.07058755, 'consistency losses/trn_torque_delta_loss': 121.76943094, 'logistical/max_param': 16.16207504, 'logistical/lr': 0.0001}
{'logistical/obs_consumed_per_second': 232.515625, 'logistical/obs_generated_per_second': 106.0546875, 'logistical/data_consumption_ratio': 2.1942939, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.01611137, 'consistency losses/trn_steer_cost': 0.03859127, 'consistency losses/trn_te_loss': 19.85157622, 'consistency losses/trn_torque_loss': 603.29908656, 'consistency losses/trn_torque_delta_loss': 127.7536754, 'logistical/max_param': 16.16288757, 'logistical/lr': 0.0001}
{'logistical/obs

KeyboardInterrupt: 

In [13]:
torch.save(m.state_dict(), f"{BESPOKE_ROOT}/models/m_{model_stem}_e74.torch")

In [10]:
%%time

eval_rw(rw_dataloader, m, wandb) # will log directly to wandb

/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/torch/utils/checkpoint.py:25: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


CPU times: user 14.3 s, sys: 417 ms, total: 14.7 s
Wall time: 14 s
